In [1]:
import json
import argparse
import torch
from transformers import MraConfig, MraForMaskedLM

No CUDA runtime is found, using CUDA_HOME='C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.3'


In [2]:
def rename_key(orig_key):
    if "model" in orig_key:
        orig_key = orig_key.replace("model.", "")
    if "norm1" in orig_key:
        orig_key = orig_key.replace("norm1", "attention.output.LayerNorm")
    if "norm2" in orig_key:
        orig_key = orig_key.replace("norm2", "output.LayerNorm")
    if "norm" in orig_key:
        orig_key = orig_key.replace("norm", "LayerNorm")
    if "transformer" in orig_key:
        layer_num = orig_key.split(".")[0].split("_")[-1]
        orig_key = orig_key.replace(f"transformer_{layer_num}", f"encoder.layer.{layer_num}")
    if "mha.attn" in orig_key:
        orig_key = orig_key.replace("mha.attn", "attention.self")
    if "mha" in orig_key:
        orig_key = orig_key.replace("mha", "attention")
    if "W_q" in orig_key:
        orig_key = orig_key.replace("W_q", "self.query")
    if "W_k" in orig_key:
        orig_key = orig_key.replace("W_k", "self.key")
    if "W_v" in orig_key:
        orig_key = orig_key.replace("W_v", "self.value")
    if "ff.0" in orig_key:
        orig_key = orig_key.replace("ff.0", "intermediate.dense")
    if "ff.2" in orig_key:
        orig_key = orig_key.replace("ff.2", "output.dense")
    if "ff" in orig_key:
        orig_key = orig_key.replace("ff", "output.dense")
    if "mlm_class" in orig_key:
        orig_key = orig_key.replace("mlm.mlm_class", "cls.predictions.decoder")
    if "mlm" in orig_key:
        orig_key = orig_key.replace("mlm", "cls.predictions.transform")
    if "backbone.backbone.encoders" in orig_key:
        orig_key = orig_key.replace("backbone.backbone.encoders", "encoder.layer")
    if "cls" not in orig_key:
        orig_key = "mra." + orig_key

    return orig_key

In [3]:
def convert_checkpoint_helper(max_position_embeddings, orig_state_dict):
    for key in orig_state_dict.copy().keys():
        val = orig_state_dict.pop(key)

        if ("pooler" in key) or ("sen_class" in key):
            continue
        else:
            orig_state_dict[rename_key(key)] = val

    orig_state_dict["cls.predictions.bias"] = orig_state_dict["cls.predictions.decoder.bias"]
    orig_state_dict["mra.embeddings.position_ids"] = torch.arange(max_position_embeddings).expand((1, -1)) + 2

    return orig_state_dict


def convert_mra_checkpoint(checkpoint_path, mra_config_file, pytorch_dump_path):
    orig_state_dict = torch.load(checkpoint_path, map_location="cpu")["model_state_dict"]
    
    with open(mra_config_file) as file:
        config_dict = json.load(file)['model']
        
    config_dict['max_position_embeddings'] = config_dict.pop('max_seq_len')
    
    config = MraConfig.from_dict(config_dict)
    print(config)
    model = MraForMaskedLM(config)

    new_state_dict = convert_checkpoint_helper(config.max_position_embeddings, orig_state_dict)

    print(model.load_state_dict(new_state_dict))
    model.eval()
    model.save_pretrained(pytorch_dump_path)

    print(f"Checkpoint successfuly converted. Model saved at {pytorch_dump_path}")

In [4]:
pmp = "D:/mra-2-4096-8-d3.model"
cfg = "D:/config_4096.json"
pdp = "C:/Users/prana/Desktop/mra-base-4096-8-d3"

In [5]:
convert_mra_checkpoint(pmp, cfg, pdp)

MraConfig {
  "approx_mode": "full",
  "attention_probs_dropout_prob": 0.1,
  "block_per_row": 8,
  "bos_token_id": 0,
  "dim": 768,
  "dropout_prob": 0.1,
  "embedding_dim": 768,
  "eos_token_id": 2,
  "head_dim": 64,
  "hidden_act": "gelu",
  "hidden_dim": 3072,
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initial_prior_diagonal_n_blocks": 1,
  "initial_prior_first_n_blocks": 3,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 4096,
  "mixed_precision": true,
  "model_type": "mra",
  "num_attention_heads": 12,
  "num_head": 12,
  "num_hidden_layers": 12,
  "num_layers": 12,
  "num_sen_type": 1,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "shared_weight": false,
  "transformers_version": "4.31.0.dev0",
  "type_vocab_size": 1,
  "vocab_size": 50265
}

<All keys matched successfully>
Checkpoint successfuly converted. Model saved at C:/Users/prana/Desktop/mra-base-4096-8-d3
